In [2]:
import json
from bs4 import Tag
from tqdm import tqdm
import zangdiyg_utils
import os
from concurrent.futures import ThreadPoolExecutor, as_completed


In [3]:

class CustomJSONEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, Tag):
            return obj.get_text()
        return str(obj)

def save_json(path, file_name, data):
    try: 
        with open(path + file_name, "w", encoding='utf-8') as outfile:
            json.dump(data, outfile, indent=4, ensure_ascii=False, cls=CustomJSONEncoder)
        print(f"Successfully saved: {file_name}")
    except Exception as e:
        print(f"Error saving {file_name}: {str(e)}")

# def read_json(path, file_name):
#     try:
#         with open(path+file_name, 'r', encoding='utf-8') as openfile:
#             Loaded_file = json.load(openfile)
#             print(f"Successfully loaded: {file_name}")
#         return Loaded_file
#     except Exception as e:
#         print(f"Error loading {file_name}: {str(e)}")
#         return None

def scrape_article(url, page_key_code):
    try:
        article_content = zangdiyg_utils.scrape_zangdiyg_article_content(url, tags=page_key_code)
        return article_content
    except Exception as e:
        print(f"Error scraping {url}: {str(e)}")
        return None

def get_content(All_links_data, Total_lenght, page_key_code, page_key_list):
    all_article = {}
    
    with ThreadPoolExecutor(max_workers=10) as executor:
        future_to_url = {}
        start = 1
        for i in range(start, Total_lenght+1):
            page_key = page_key_code + str(i)
            all_link_page = All_links_data[page_key]["Links"]
            
            for url in all_link_page:
                future = executor.submit(scrape_article, url, page_key_code)
                future_to_url[future] = (page_key, url)
        
        for future in tqdm(as_completed(future_to_url), total=len(future_to_url)):
            page_key, url = future_to_url[future]
            try:
                article_content = future.result()
                if article_content:
                    article_key = f"{page_key}_kbcmw_Article_{len([k for k in all_article if k.startswith(page_key)]) + 1}"
                    all_article[article_key] = article_content
            except Exception as e:
                print(f"Error processing {url}: {str(e)}")
    
    Failure_count = sum(1 for article in all_article.values() if article["Response"] != 200)
    print(f"Total Failure in the {page_key_list[1]} article: {Failure_count}")
    
    save_file_name = f"zangdiyg_ALL_content_{page_key_list[1]}.json"
    print(save_file_name)
    path = "./data/parallel_content/"
    save_json(path, save_file_name, all_article)

def process_json_file(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            All_links_data = json.load(file)
            Total_lenght = len(All_links_data)
            print(f"Total page in {os.path.basename(file_path)}: {Total_lenght}")
        
            print(f"page key name: {list(All_links_data.keys())[-1]}")
            page_key_list = list(All_links_data.keys())[-1].split(" ")
            
            page_key_code = "Page "+page_key_list[1]+" "
            print(f"Page key code: {page_key_code}")
            get_content(All_links_data, Total_lenght, page_key_code, page_key_list)
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON in file {os.path.basename(file_path)}: {str(e)}")
    except Exception as e:
        print(f"Error processing file {os.path.basename(file_path)}: {str(e)}")

def get_json_files(directory):
    json_files = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.json')]
    
    with ThreadPoolExecutor(max_workers=10) as executor:
        list(tqdm(executor.map(process_json_file, json_files), total=len(json_files)))
    
    print(f"Processed {len(json_files)} files")


In [ ]:
%%time
# Example usage
directory_path = './data/links/'
get_json_files(directory_path)

Total page in zangdiyg_ALL_link_རྒྱལ་སྤྱི་གསར་འགྱུར།.json: 76
page key name: Page རྒྱལ་སྤྱི་གསར་འགྱུར། 76
Page key code: Page རྒྱལ་སྤྱི་གསར་འགྱུར། 
Total page in zangdiyg_ALL_link_རྒྱལ་ནང་གསར་འགྱུར།.json: 198
page key name: Page རྒྱལ་ནང་གསར་འགྱུར། 198
Page key code: Page རྒྱལ་ནང་གསར་འགྱུར། 
Total page in zangdiyg_ALL_link_སི་ཁྲོན་གསར་འགྱུར།.json: 213
page key name: Page སི་ཁྲོན་གསར་འགྱུར། 213
Page key code: Page སི་ཁྲོན་གསར་འགྱུར། 
Total page in zangdiyg_ALL_link_བོད་འབྲེལ་ས་ཁུལ།.json: 185
page key name: Page བོད་འབྲེལ་ས་ཁུལ། 185
Page key code: Page བོད་འབྲེལ་ས་ཁུལ། 
Total page in zangdiyg_ALL_link_སྒྲུང་གཏམ།.json: 4
page key name: Page སྒྲུང་གཏམ། 4
Page key code: Page སྒྲུང་གཏམ། 
Total page in zangdiyg_ALL_link_ལྷུག་རྩོམ།.json: 10
page key name: Page ལྷུག་རྩོམ། 10
Page key code: Page ལྷུག་རྩོམ། 
Total page in zangdiyg_ALL_link_སྙན་རྩོམ།.json: 18
page key name: Page སྙན་རྩོམ། 18
Page key code: Page སྙན་རྩོམ། 
Total page in zangdiyg_ALL_link_བསྒྱུར་རྩོམ།.json: 4
page key name: Page བསྒྱ

  0%|          | 0/54 [00:00<?, ?it/s]

  0%|          | 0/191 [00:00<?, ?it/s]


  0%|          | 0/116 [00:00<?, ?it/s]



  0%|          | 0/295 [00:00<?, ?it/s]




  0%|          | 0/670 [00:00<?, ?it/s]IOStream.flush timed out






  0%|          | 0/370 [00:00<?, ?it/s]




  1%|          | 1/121 [00:27<54:39, 27.33s/it]t]


  1%|          | 1/116 [00:19<37:02, 19.33s/it]




  2%|▏         | 2/121 [00:27<22:41, 11.44s/it]t]



  0%|          | 1/295 [00:17<1:26:11, 17.59s/it]


  2%|▏         | 2/116 [00:24<15:25,  8.12s/it]





  0%|          | 1/370 [00:11<21:59,  3.58s/it]





  1%|          | 2/370 [00:12<40:50,  6.66s/it]




  1%|          | 5/670 [00:37<23:27,  2.12s/it]  




  1%|          | 6/670 [00:47<1:30:13,  8.15s/it]IOStream.flush timed out




  1%|          | 2/295 [01:16<49:45, 10.19s/it]  



  7%|▋         | 8/121 [01:48<03:38,  1.93s/it]t]





  1%|          | 3/370 [01:42<32:41,  5.34s/it]

  1%|          | 1/191 [02:39<1:07:30, 21.32s/it]




  1%|  

Total Failure in the སྒྲུང་གཏམ། article: 121
zangdiyg_ALL_content_སྒྲུང་གཏམ།.json


  0%|          | 3/6210 [00:05<2:04:42,  1.21s/it]







  0%|          | 4/6210 [00:06<3:09:36,  1.83s/it]









Successfully saved: zangdiyg_ALL_content_སྒྲུང་གཏམ།.json
Total page in zangdiyg_ALL_link_རི་མོ།.json: 4
page key name: Page རི་མོ། 4
Page key code: Page རི་མོ། 


...


  2%|▏         | 1/54 [1:42:23<90:26:50, 6143.60s/it]







 38%|███▊      | 2382/6210 [1:34:29<1:12:28,  1.14s/it]









 75%|███████▍  | 847/1133 [26:23<08:44,  1.84s/it]s/it]









 75%|███████▍  | 848/1133 [26:24<06:31,  1.38s/it]s/it]








 75%|███████▍  | 849/1133 [26:25<06:32,  1.38s/it]s/it]









 75%|███████▌  | 852/1133 [26:27<04:13,  1.11it/s]s/it]







 38%|███▊      | 2383/6210 [1:34:34<2:19:30,  2.19s/it]









 31%|███       | 2423/7773 [1:34:30<2:30:31,  1.69s/it]








 34%|███▎      | 2386/7074 [1:34:32<2:00:29,  1.54s/it]







 38%|███▊      | 2384/6210 [1:34:34<1:43:48,  1.63s/it]








 34%|███▎      | 2387/7074 [1:34:32<1:39:57,  1.28s/it]









 31%|███       | 2424/7773 [1:34:31<1:58:38,  1.33s/it]








 75%|███████▌  | 853/1133 [26:29<05:44,  1.23s/it]s/it]







 38%|███▊      | 2385/6210 [1:34:35<1:41:20,  1.59s/it]









 75%|███████▌  | 854/1133 [26:30<04:42,  1.01s/it]s/it]








 75%|███████▌  | 855/1133 [26:31<04:07,

Total Failure in the བདེ་ཐང་ཤེས་བྱ། article: 1110
zangdiyg_ALL_content_བདེ་ཐང་ཤེས་བྱ།.json











 38%|███▊      | 2666/7074 [1:39:42<1:09:24,  1.06it/s]








 38%|███▊      | 2667/7074 [1:39:43<52:56,  1.39it/s]  








 38%|███▊      | 2668/7074 [1:39:43<39:27,  1.86it/s]

Successfully saved: zangdiyg_ALL_content_བདེ་ཐང་ཤེས་བྱ།.json










 43%|████▎     | 2667/6210 [1:39:45<1:01:48,  1.05s/it]









 35%|███▍      | 2698/7773 [1:39:42<1:17:29,  1.09it/s]








 38%|███▊      | 2669/7074 [1:39:43<40:47,  1.80it/s]







 43%|████▎     | 2668/6210 [1:39:46<53:45,  1.10it/s]  









 35%|███▍      | 2699/7773 [1:39:43<1:19:55,  1.06it/s]







 43%|████▎     | 2669/6210 [1:39:47<51:50,  1.14it/s]








 38%|███▊      | 2670/7074 [1:39:45<59:33,  1.23it/s]







 43%|████▎     | 2670/6210 [1:39:47<43:16,  1.36it/s]








 38%|███▊      | 2671/7074 [1:39:45<52:26,  1.40it/s]







 43%|████▎     | 2671/6210 [1:39:47<35:41,  1.65it/s]









 35%|███▍      | 2700/7773 [1:39:44<1:26:05,  1.02s/it]







 43%|████▎     | 2672/6210 [1:39:47<28:29,  2.07it/s]







 43%|████▎     | 2673/6210 [1:39:48<25:13,  2.34it/s]








 38%|███▊      | 2672/7074 [1:39:46<47:53,  1.53it/s]







 43%|████▎     | 2674/6210 [1:39:48<24:43,  2.38it/s]









 35%|███▍      | 2701/7773 [1:39:45<1:20:55,  1.04it/s]

In [ ]:
# 14:44

In [ ]:
json_files = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.json')]
json_files